# polytop
Python library for constructing polymer toplogies

### import the library and confirm version

In [1]:
from polytop import polytopVer
polytopVer()

0.1

In [4]:
# Load sample ITPs
from polytop import Molecule 
GLU = Molecule.fromITP('test/samples/glutamine.itp')
ARG = Molecule.fromITP('test/samples/arginine.itp')

In [5]:
# render display presentation
GLU

In [6]:
#render display presentation
ARG

In [8]:
#convert into mononmers by specifying RHS and LHS excision points
GLU_monomer = Monomer(molecule=GLU,LHS=2,RHS=20)
ARG_monomer = Monomer(molecule=ARG,LHS=91,RHS=1)

NameError: name 'Monomer' is not defined

In [7]:
# orchestrate polymerization
GAAAG = Polymer(distributePartialCharges=ACROSSMONOMER)+GLU_monomer+ARG_monomer+ARG_monomer+ARG_monomer+GLU_monomer 
GAAAG.toITP("tests/samples/GAAAG.itp")
GAAAG

NameError: name 'Polymer' is not defined

In [ ]:
%%bash
jupyter nbconvert README.ipynb --to markdown